In [1]:
// import { ChatOpenAI } from "@langchain/openai";
// import { HumanMessage } from "@langchain/core/messages";

// const model = new ChatOpenAI();

// await model.invoke([
//     new HumanMessage("Tell me a joke")
// ])


: 

In [2]:
// import { StringOutputParser } from "@langchain/core/output_parsers";
// import { HumanMessage } from "@langchain/core/messages";

// const parser = new StringOutputParser();
// const model = new ChatOpenAI();

// const chain = model.pipe(parser)

// await chain.invoke([
//     new HumanMessage("Tell me a joke")
// ])


In [6]:
// StructuredOutputParser
import { StructuredOutputParser, OutputFixingParser } from "langchain/output_parsers";
import { PromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { Ollama } from "@langchain/community/llms/ollama";
import { z }  from "zod";
const schema = z.object({
    answer: z.string().describe("用户问题的答案"),
    confidence: z.number().min(0).max(100).describe("问题答案的可信度评分，满分 100")
})
const parser = StructuredOutputParser.fromZodSchema(schema);
// const parser = StructuredOutputParser.fromNamesAndDescriptions({
//     answer: "用户问题的答案",
//     evidence: "你回答用户问题所依据的答案",
//     confidence: "问题答案的可信度评分，格式是百分数"
// })
// console.log(parser.getFormatInstructions())
const prompt = PromptTemplate.fromTemplate("尽可能的回答用的问题 \n{instructions} \n{question}")
// const model = new ChatOpenAI();
const model = new Ollama({
    baseUrl: "http://localhost:11434", 
    model: "llama3", 
  });
const chain = prompt.pipe(model).pipe(parser)
const res = await chain.invoke({
    question: "湖北省的省会城市是什么？",
    instructions: parser.getFormatInstructions()
})
// console.log(res, 'res');
const wrongOutput = {
    "answer": "湖北省的省会城市是襄阳",
    "confidence": 90, 
}
const fixParser = OutputFixingParser.fromLLM(model, parser);
const output = await fixParser.parse(JSON.stringify(wrongOutput))
console.log(output)


{ answer: "湖北省的省会城市是襄阳", confidence: 90 }


In [ ]:
// List Output Parser
import { CommaSeparatedListOutputParser } from "@langchain/core/output_parsers";
const parser = new CommaSeparatedListOutputParser()
// console.log(parser.getFormatInstructions());
const model = new Ollama({
    baseUrl: "http://localhost:11434", 
    model: "llama3", 
  });
const prompt = PromptTemplate.fromTemplate("列出3个 {country} 的著名的互联网公司.\n{instructions}")
const chain = prompt.pipe(model).pipe(parser)
const response = await chain.invoke({
    country: "China",
    instructions: parser.getFormatInstructions()
})
console.log(response);


[
  "Here are 3 famous Chinese internet companies:\n\nAlibaba",
  "Tencent",
  "JD.com"
]


: 